In [ ]:
# TODO: get parameter count & break down by layer
# TODO: get VRAM usage for empty, full seq, grad states

# TODO: train standard Transformer on entropy dataset
# TODO: plot loss and weight checkpoints
# TODO: then introduce RL

# TODO: get first 30 MB of tiny-stories dataset

# TODO(opt): use tiktoken 

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.cuda.is_available()


if torch.cuda.is_available():
    # Get free and total memory in bytes
    free_memory_bytes, total_memory_bytes = torch.cuda.mem_get_info()

    # Convert to GB for better readability
    free_memory_gb = free_memory_bytes / (1024**3)
    total_memory_gb = total_memory_bytes / (1024**3)

    print(f"Total GPU VRAM: {total_memory_gb:.2f} GB")
    print(f"Free GPU VRAM: {free_memory_gb:.2f} GB")
else:
    print("CUDA is not available. No GPU found.")

CUDA is not available. No GPU found.


In [21]:
# dataset loading
with open('basic_entropy_dataset_2.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    print(text[:209])

478947894789478947894789478947894789478947894789478947894789478947894789478947894789478947894789478
145914591459145914591459145914591459145914591459145914591459145914591459145914591459145914591459145
025602560


In [ ]:
# Hyperparameters

context_length = 100     # Max input length
batch_size = 32         # how many examples in one forward pass
embedding_dim = 12     # len of each vector representing a token
num_heads = 4           # number of attention heads in each layer
num_layers = 3          # number of transformer blocks(each = attention+MLP)
dropout = 0.1
num_epochs = 10         # how many times go through the entire dataset
lr = 1e-3

In [46]:
# Build vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

# Encoding functions
def encode(s): return [stoi[c] for c in s]
def decode(l): return ''.join([itos[i] for i in l])

print(encode("\n0123456789"))
print("vocab size: ", vocab_size)

print(decode([3, 4, 5, 6, 7, 8, 9, 10]))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
vocab size:  11
23456789


In [60]:
# Convert the full dataset (which is a string of characters) into a tensor of integers(idx of chars in vocab)
# e.g: "hello" → [7, 4, 11, 11, 14] → tensor([7, 4, 11, 11, 14])
data = torch.tensor(encode(text), dtype=torch.long)


# get a line, give each char positional encoding, then sample 

def get_batch(split='train'):
    # Randomly samples batch_size starting indices (integers) from 0 to (len(data)-context_length)
    # Each index is the start of a training sequence of length context_length.
    # Subtracting context_length ensures you don’t run past the end of the data when slicing.
    example_seeds = torch.randint(0,100, (1,))

    x = torch.stack([data[(i*100):(i*100)+context_length] for i in example_seeds])
    y = torch.stack([data[(i*100)+1:(i*100)+context_length+1] for i in example_seeds])

    print("X:", x)
    print("Y:", y)
    return x, y
    # return x.to(device), y.to(device) #Given: "the ca", Predict: "he cat"

x= get_batch()

print("X- :", x[0][0])
print("Y- :", x[1])


print("X shape:", x[0].shape)
print("Y shape:", x[1].shape)

print("X:", decode(x[0][0].tolist()))
print("Y:", decode(x[1][0].tolist()))


X: tensor([[3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9,
         3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9,
         3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9]])
Y: tensor([[5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3,
         5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3,
         5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3]])
X- : tensor([3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9,
        3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9,
        3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9])
Y- : tensor([[5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3,
         5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3,
         5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3, 5, 6, 9, 3]])
X shape: torch.Size([1, 64])
Y shape: torch.Size([1, 64])
X: 2458245824582458245824582458245824582458245

In [6]:
class SelfAttentionHead(nn.Module):

    # embedding_dim: input dimension for each token (e.g 128)
    # head_size: output dimension of this head (e.g., 128 // num_heads_being_4 = 32)
    def __init__(self, embedding_dim, head_size):
        super().__init__()

        # initialize single-layer fully-connected neural network
        #                     in_features ,  out_features  , bias=false
        self.key = nn.Linear(embedding_dim, head_size, bias=False)
        self.query = nn.Linear(embedding_dim, head_size, bias=False)
        self.value = nn.Linear(embedding_dim, head_size, bias=False)

        # Creates a lower triangular matrix of shape [context_length, context_length]
        # Used for causal masking: tokens can only attend to earlier or same-position tokens, never future tokens
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length))) 

        # Dropout layer for regularizing attention weights, prevents overfitting.
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape # batch size, time steps (context length), channels (embedding_dim)

        # shape [B, T, head_size]
        k = self.key(x)
        q = self.query(x)

        # Computes attention scores: dot product between each query and all keys.
        # Shape becomes [B, T, T]: every token attends to every other token
        wei = q @ k.transpose(-2, -1) / (C ** 0.5) # Divide by sqrt(C) to stabilize gradients
        
        # Apply the causal mask: upper triangle is -inf → zero probability after softmax
        # so that position t can’t see future positions t+1 and beyond.
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 

        # Turns the attention scores into a probability distribution over past tokens
        wei = F.softmax(wei, dim=-1)

        # Dropout on attention weights (regularization).
        wei = self.dropout(wei)

        # Compute the weighted average of the values for each position
        v = self.value(x)
        return wei @ v # Output shape is [B, T, head_size]



class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super().__init__()

        # Splits the full embedding into multiple smaller "heads" that run attention independently. 
        # This allows the model to learn multiple types of relationships in parallel
        head_size = embedding_dim // num_heads

        # Creates num_heads attention heads. 
        # Each head processes the input differently with its own Q, K, V weight matrices.
        self.heads = nn.ModuleList([SelfAttentionHead(embedding_dim, head_size) for _ in range(num_heads)])

        # After heads are concatenated, project back to the original embedding dimension. 
        # This mixes information across heads.
        self.proj = nn.Linear(embedding_dim, embedding_dim)

        # Dropout after projection — again, to regularize.
        self.dropout = nn.Dropout(dropout)    

    def forward(self, x):

        # Run all heads independently and concatenate their outputs along the feature dimension.
        out = torch.cat([h(x) for h in self.heads], dim=-1)

        # Apply the projection + dropout to mix and regularize the combined multi-head output.
        return self.dropout(self.proj(out))
    


# A standard 2-layer MLP(Input → expand → ReLU → compress → Dropout).
# Applies to each token independently (position-wise).
class FeedForward(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_dim, 4 * embedding_dim),
            nn.ReLU(),
            nn.Linear(4 * embedding_dim, embedding_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)



class DecoderBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super().__init__()
        self.sa = MultiHeadAttention(embedding_dim, num_heads)
        self.ffwd = FeedForward(embedding_dim)
        self.ln1 = nn.LayerNorm(embedding_dim)
        self.ln2 = nn.LayerNorm(embedding_dim)

    def forward(self, x):
        #  Applies layer norm, then attention. Adds the result back to x → residual connection.   
        x = x + self.sa(self.ln1(x))

        # Same idea: norm → feedforward → residual
        x = x + self.ffwd(self.ln2(x))

        # Long-range dependencies (via attention)
        # Complex feature transformations (via MLP)
        # Stable, deep learning (via residuals + norm)
        return x


In [7]:

# This is your complete transformer model(input embeddings(token+pos), 3 transformer blocks, output layer)
class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()

        # This allows tokens to be transformed into dense vectors the model can work with.
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)

        #TODO: RoPE
        # Adds information about token position in the sequence, attention has no sense of order ! 
        # Each position 0 to context_length-1 gets a learnable vector of same size as token embedding.
        self.position_embedding = nn.Embedding(context_length, embedding_dim)

        # Builds a stack of num_layers decoder blocks. 
        # Each block includes multi-head attention, feedforward, layer norm, and residuals
        self.blocks = nn.Sequential(*[
            DecoderBlock(embedding_dim, num_heads) for _ in range(num_layers)
        ])

        # Final layer normalization after all blocks.
        # Helps stabilize activations and gradients before output.
        self.ln_f = nn.LayerNorm(embedding_dim)

        # Maps the output of each token (which is a vector) to logits over the vocabulary.
        self.head = nn.Linear(embedding_dim, vocab_size)



    # idx: input token IDs of shape [B, T] (batch size × context length)
    # targets: next-token targets (same shape), optional (used for inference)
    def forward(self, idx, targets=None):

        B, T = idx.shape # B: batch size,  T: time steps (context length)


        # Shape becomes [B, T, embedding_dim]
        tok_emb = self.token_embedding(idx) #  Each token is mapped to its learned vector

        # Shape: [T, embedding_dim]
        # Each position 0...T-1 gets a learnable vector
        pos_emb = self.position_embedding(torch.arange(T, device=device)) 
        # Broadcasted to match batch size

        # Shape: [B, T, embedding_dim]
        # Add both embeddings together elementwise, each token has a sense of identity and order
        x = tok_emb + pos_emb 


        # Passes the full sequence through the stack of decoder blocks(3 in this case)
        # Each block refines the representation of each token
        x = self.blocks(x)

        # Apply layer normalization at the end for stability.
        # Improves convergence and output quality
        x = self.ln_f(x)

        # Shape: [B, T, vocab_size]
        # For each position in the sequence, you get a score for each possible next token.
        logits = self.head(x) 


        # If no labels are provided, just return the logits (e.g., for sampling).
        if targets is None:
            return logits, None


        # Flatten logits and targets: from [B, T, vocab_size] and [B, T] to [B*T, ...]
        # Compute cross-entropy loss
        # Teaches the model to assign high probabilities to the correct next characters.
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

        # Return both predictions and the loss (for backpropagation during training).
        return logits, loss


    # idx: a starting sequence of token IDs (e.g., [[12]])
    # max_new_tokens: how many new tokens to append
    def generate(self, idx, max_new_tokens):

        # Loop to generate one token at a time, up to max_new_tokens.
        for _ in range(max_new_tokens):

            # The model can only handle inputs up to that length, so
            # Truncate input to most recent context_length tokens
            idx_cond = idx[:, -context_length:]

            # Pass current context into the model, no loss just logits
            logits, _ = self(idx_cond)

            # Take the logits for the last position in the sequence.
            # Softmax turns it into a probability distribution over the vocabulary.
            probs = F.softmax(logits[:, -1, :], dim=-1)

            # Sample from the probability distribution. + a little randomness for creativity 
            next_token = torch.multinomial(probs, num_samples=1)

            # Add the sampled token to the end of the input.
            # The loop continues with the longer sequence.
            idx = torch.cat([idx, next_token], dim=1)

        # The final sequence includes the original input + all generated tokens.
        return idx

In [ ]:
# Training loop

"""

    🧠 The model gets a random chunk of training data.

    🎯 It tries to predict the next token in each sequence.

    ❌ It measures how wrong it was (loss).

    📉 It computes how to adjust weights (gradients).

    🔧 It updates the weights slightly to improve.

    📈 Over time, the model becomes better at next-token prediction.

"""




# Creates an instance of your Transformer model and moves it to the GPU (or CPU) based on availability
# All your tensors (x, y) and model parameters must
#   live on the same device (GPU if available). to(device) ensures that
model = TransformerLanguageModel().to(device)

# Creates the AdamW optimizer (Adam + weight decay) for updating model parameters.
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

# Repeats the training process num_epochs times (complete passes over the data).
for epoch in range(num_epochs):

    # Some layers like Dropout and LayerNorm behave differently 
    #   in training vs inference (e.g., dropout is disabled in .eval() mode).
    model.train()

    # Grabs one batch of training examples
    #   xb = input tensor (token sequences)
    #   yb = target tensor (the correct next characters)
    xb, yb = get_batch()

    # Pass the inputs (xb) through the model to get:
    #   logits: the raw predicted scores (before softmax)
    #   loss: cross-entropy loss comparing logits to yb
    logits, loss = model(xb, yb)

    print(logits.shape) # [batch_size, context_length, vocab_size]
    print(loss.shape)

    # Gradients accumulate by default in PyTorch. If you don’t
    #   clear them, you’ll be mixing gradients from multiple steps 
    #   which leads to incorrect updates
    optimizer.zero_grad()

    # This is automatic differentiation. PyTorch builds a computation graph as you go,
    #    and this line applies backpropagation
    # Computes the gradients of the loss w.r.t. all model parameters
    loss.backward()

    # Updates the model’s parameters using the gradients
    optimizer.step()

    # Logs the current epoch number and the loss value. for monitoring
    print(f"Epoch {epoch} | Loss: {loss.item():.4f}")

torch.Size([32, 64, 65])
torch.Size([])
Epoch 0 | Loss: 4.2929
torch.Size([32, 64, 65])
torch.Size([])
Epoch 1 | Loss: 3.9854
torch.Size([32, 64, 65])
torch.Size([])
Epoch 2 | Loss: 3.7052
torch.Size([32, 64, 65])
torch.Size([])
Epoch 3 | Loss: 3.5470
torch.Size([32, 64, 65])
torch.Size([])
Epoch 4 | Loss: 3.5303
torch.Size([32, 64, 65])
torch.Size([])
Epoch 5 | Loss: 3.3733
torch.Size([32, 64, 65])
torch.Size([])
Epoch 6 | Loss: 3.3404
torch.Size([32, 64, 65])
torch.Size([])
Epoch 7 | Loss: 3.3467
torch.Size([32, 64, 65])
torch.Size([])
Epoch 8 | Loss: 3.2707
torch.Size([32, 64, 65])
torch.Size([])
Epoch 9 | Loss: 3.3032


In [ ]:
"""

What's in the .pt file?
    It stores a mapping like:

{
  'token_embedding.weight': Tensor([...]),
  'blocks.0.sa.heads.0.key.weight': Tensor([...]),
  ...
}

Each tensor is a learned weight or bias from your model's layers.
"""




# Save weights
# model.state_dict() returns a Python dictionary 
#   that holds all the learned weights (parameters) of your model.
# torch.save(...) writes that dictionary to disk as a file.
torch.save(model.state_dict(), "transformer_weights.pt")


# Load weights
# Creates a new instance of your model. move it to GPU
model = TransformerLanguageModel().to(device)

# torch.load(...) reads the .pt file from disk and returns the dictionary of saved weights.
# model.load_state_dict(...) copies those weights into your new model instance
model.load_state_dict(torch.load("transformer_weights.pt"))

# Sets the model to inference mode.
# Certain layers behave differently in .train() vs .eval():
    #Dropout is disabled (no random masking)
    #LayerNorm/BatchNorm uses learned averages instead of batch stats
model.eval()

TransformerLanguageModel(
  (token_embedding): Embedding(65, 128)
  (position_embedding): Embedding(64, 128)
  (blocks): Sequential(
    (0): DecoderBlock(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x SelfAttentionHead(
            (key): Linear(in_features=128, out_features=32, bias=False)
            (query): Linear(in_features=128, out_features=32, bias=False)
            (value): Linear(in_features=128, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): LayerNorm((128,), eps=1e-05, eleme

In [11]:
# sample inference

# Converts the single character 'H' into its token ID using the character-to-index map stoi.
# Wraps it in two layers of brackets: [[...]], making it shape [1, 1], which is:
    # Batch size = 1
    # Sequence length = 1
#       because Transformers expect input as a 2D tensor: [batch_size, context_length]
# Converts it into a torch.Tensor of type long (required for indexing into embeddings)
# Moves it to the same device as your model (GPU or CPU).
context = torch.tensor([[stoi['H']]], dtype=torch.long, device=device)


"""
The generate() function:

    Truncates the input to the max context length

    Gets logits from the model

    Converts logits to probabilities

    Samples the next token from those probabilities

    Appends the token to the sequence

    Repeats
"""
generated = model.generate(context, max_new_tokens=100)

# turns your generated tensor of integers into human-readable text
# generated[0]: Selects the first (and only) sequence in the batch.
# .tolist(): Converts the tensor (e.g. tensor([34, 17, 53, ...])) to a Python list.
# decode(...): Converts list of token IDs back into a string using your itos (index-to-symbol) mapping.
print(decode(generated[0].tolist()))

Hne'bxueoed Dienheltzs
eFseDdTnc
t ahIuat,np uwernf;th  ne mhr eyhUarus
iRFh hi lict,IUUioasnAi Oeo t
